ใน ep ที่แล้ว 

# 0. Install

Install Library ที่จำเป็น

In [83]:
## Colab
! curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import

Import Library ที่จะใช้ ในที่นี้คือ fastai และ fastai.text 

In [0]:
from fastai import *
from fastai.text import *

In [0]:
# import fastai.utils.collect_env
# fastai.utils.collect_env.show_install()

# 2. Dataset

ในเคสนี้เราจะใช้ IMDB Movie Review เป็นรีวิวหนังจากเว็บ IMDB ที่มีข้อความ และ คะแนนว่าเป็นแง่บวก หรือแง่ลบ เหมือนใน ep ก่อน ๆ 

ในการพัฒนา เราจะใช้ Dataset ชุดเล็กก่อน จะได้เร็ว เมื่อเทสทุกอย่างเรียบร้อย แล้วจึงขยับไปใช้ Dataset ชุดเต็ม 

In [87]:
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

[PosixPath('/content/data/imdb_sample/texts.csv')]

# 3. Preprocessing

## 3.1 Data Block API

เราจะใช้ [Data Block API](https://www.bualabs.com/archives/2693/data-block-api-data-pipeline-machine-learning-supervised-learning-preprocessing-ep-5/) กำหนด Data Pipeline ทีละขั้น Tokenization, Numericalization, Split, Labeling ก็จะทำให้ยืดหยุ่นมากขึ้น

In [0]:
data = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0)
                .databunch())

In [93]:
data.vocab.itos[:20]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i']

In [94]:
data.train_ds[0][0]

Text xxbos xxmaj un - xxunk - believable ! xxmaj meg xxmaj ryan does n't even look her usual xxunk lovable self in this , which normally makes me forgive her shallow xxunk acting xxunk . xxmaj hard to believe she was the producer on this dog . xxmaj plus xxmaj kevin xxmaj kline : what kind of suicide trip has his career been on ? xxmaj xxunk ... xxmaj xxunk ! ! ! xxmaj finally this was directed by the guy who did xxmaj big xxmaj xxunk ? xxmaj must be a replay of xxmaj jonestown - hollywood style . xxmaj xxunk !

In [95]:
data.train_ds[0][0].data[:10]

array([   2,    5, 4619,   25,    0,   25,  867,   52,    5, 3776])

# 2/2. Full Dataset

In [96]:
path = untar_data(URLs.IMDB)
path.ls()

[PosixPath('/content/data/imdb/lm_databunch'),
 PosixPath('/content/data/imdb/data_save.pkl'),
 PosixPath('/content/data/imdb/train'),
 PosixPath('/content/data/imdb/test'),
 PosixPath('/content/data/imdb/unsup'),
 PosixPath('/content/data/imdb/README'),
 PosixPath('/content/data/imdb/tmp_clas'),
 PosixPath('/content/data/imdb/lm_databunch.pkl'),
 PosixPath('/content/data/imdb/tmp_lm'),
 PosixPath('/content/data/imdb/imdb.vocab')]

In [97]:
(path/'train').ls()

[PosixPath('/content/data/imdb/train/pos'),
 PosixPath('/content/data/imdb/train/labeledBow.feat'),
 PosixPath('/content/data/imdb/train/neg'),
 PosixPath('/content/data/imdb/train/unsupBow.feat')]

# 3/2. Preprocessing

Data Pipeline ด้วย Data Block API

In [0]:
# bs=48
# bs=24
bs=192

In [0]:
data_lm = (TextList.from_folder(path)
                    .filter_by_folder(include=['train', 'test', 'unsup'])
                    .split_by_rand_pct(0.1)
                    .label_for_lm()
                    .databunch(bs=bs, num_workers=1))

มีคำศัพท์ 60,000 คำ ข้อมูล 90,000 Record

In [100]:
len(data_lm.vocab.itos), len(data_lm.train_ds)

(60000, 90000)

In [101]:
data_lm.show_batch()

idx,text
0,"a dream job at a new employment agency in xxmaj london , whose sinister representative seems very anxious to ascertain if she has living relatives or a boyfriend . xxmaj after reporting to duty , she wakes up ( xxmaj having xxmaj been xxmaj drugged ) in a vast xxmaj manderley - like pile on the xxmaj cornish coast , supposedly as the barmy - in - the - crumpet"
1,"marshmorton ( xxmaj fontaine ) . xxmaj the tale is complicated by a betting pool among the xxmaj marshmorton servants that is run by ( and rigged for ) head butler xxmaj keggs ( xxmaj gardiner ) , who 's betting on xxmaj lady xxmaj alice 's cousin xxmaj reggie ( xxmaj noble ) , the favorite of xxmaj alice 's stuffy , domineering aunt ( xxmaj collier ) ."
2,"close . \n \n i think western audiences will be very confused with 2 scenes one of which involves a monk trying to burn himself alive and the other concerning the villagers chanting that it is the end of the world . xxmaj the mentioned scenes are derived from certain interpretations of xxmaj xxunk xxmaj buddhist text ( xxmaj xxunk xxmaj buddhism can be found in xxmaj china ,"
3,"on all the vile scumbags who make poor xxmaj eddie 's life the proverbial living hell , only to have meek xxmaj eddie prove to be a most reluctant would - be accomplice . xxmaj it 's up to xxmaj eddie , assisted by token nice girl xxmaj leslie xxmaj graham ( likeably essayed by the lovely xxmaj lisa xxmaj xxunk ) , to stop xxmaj sammi before things get"
4,". xxmaj overall , i recommend this movie and if you enjoy the topic of school aggression and violence and learning more about it , i also suggest the documentary "" xxmaj it 's a xxmaj girls xxmaj world "" put out by xxup cbc in 2004 , which deals with the topic of social bullying , comparing and contrasting two groups of girls one in xxmaj montreal , xxmaj"


เซฟไว้ก่อน 

In [0]:
data_lm.save('lm_databunch')

โหลด Language Model DataBunch ที่เซฟไว้ แล้วเช็คข้อมูล

In [119]:
data_lm = load_data(path, 'lm_databunch', bs=bs)
len(data_lm.vocab.itos), len(data_lm.train_ds)

(60000, 90000)

# 4. Language Model จาก WikiText-103

สร้าง Language Model Learner ขึ้นมา โดยใช้ data_lm จาก IMDB ด้านบน และ สถาปัตยกรรม AWD_LSTM Architecture

AWD_LSTM ได้ถูก Pre-Train กับ WikiText-103 Dataset เรียบร้อยแล้ว

In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [106]:
Config().model_path().ls()

[PosixPath('/root/.fastai/models/wt103-fwd.tgz'),
 PosixPath('/root/.fastai/models/wt103-fwd')]

In [107]:
(Config().model_path()/'wt103-fwd').ls()

[PosixPath('/root/.fastai/models/wt103-fwd/lstm_fwd.pth'),
 PosixPath('/root/.fastai/models/wt103-fwd/itos_wt103.pkl')]

## 4.1 vocab Dictionary ของ WikiText-103 

โหลด Dictionary ของ WikiText-103 ขึ้นมา

In [0]:
wiki_itos = pickle.load(open(Config().model_path()/'wt103-fwd/itos_wt103.pkl', 'rb'))

ดูจำนวนคำศัพท์ ตัวอย่างข้อมูล หัว / ท้าย

In [109]:
len(wiki_itos), wiki_itos[:20], wiki_itos[-20:]

(60000,
 ['xxunk',
  'xxpad',
  'xxbos',
  'xxeos',
  'xxfld',
  'xxmaj',
  'xxup',
  'xxrep',
  'xxwrep',
  'the',
  ',',
  '.',
  'of',
  'and',
  'in',
  'to',
  'a',
  '=',
  '"',
  'was'],
 ['repaying',
  'ak-47',
  'cultic',
  'jaume',
  'leonis',
  'subtracting',
  'sexed',
  'bursa',
  '-style',
  'lithographs',
  'wgbh',
  'subsystem',
  '831',
  'lifelines',
  'bergeron',
  'adaptions',
  'slurs',
  'acasta',
  'stuccoed',
  'spellbound'])

## 4.2 vocab Dictionary ของ IMDB

เปรียบเทียบ 2 vocab Dictionary

In [0]:
vocab = data_lm.vocab

In [0]:
imdb_itos = data_lm.vocab.itos

In [132]:
vocab.stoi['love']

134

In [133]:
imdb_itos[vocab.stoi['love']]

'love'

In [134]:
imdb_itos[vocab.stoi['Sukhumvit']]

'xxunk'

In [0]:
awd = learn_lm.model[0]

In [0]:
from scipy.spatial.distance import cosine as dist

In [0]:
enc = learn_lm.model[0].encoder

In [138]:
enc.weight.size()

torch.Size([60000, 400])

## 4.3 Difference in vocab between IMDB and Wikipedia

ขนาด vocab เท่ากัน

In [139]:
len(wiki_itos), len(imdb_itos)

(60000, 60000)

แต่ข้างใน ไม่เหมือนกัน เปรียบเทียบ 10 คำศัพท์สุดท้าย

In [140]:
wiki_itos[-10:], imdb_itos[-10:]

(['wgbh',
  'subsystem',
  '831',
  'lifelines',
  'bergeron',
  'adaptions',
  'slurs',
  'acasta',
  'stuccoed',
  'spellbound'],
 ['steadiness',
  'shimono',
  'enuff',
  'simper',
  'spookiest',
  "o'daniel",
  'skinflint',
  'facelift',
  'slates',
  'hospitalised'])

In [0]:
wiki_words = set(wiki_itos)
imdb_words = set(imdb_itos)

In [148]:
wiki_not_imdb = wiki_words.difference(imdb_words)
len(wiki_not_imdb), [val for i, val in enumerate(itertools.islice(wiki_not_imdb, 20))]

(24264,
 ['forestry',
  'dunstan',
  'phineas',
  'asser',
  'aylwin',
  'prospectus',
  'pausanias',
  '148',
  'neymar',
  'sectional',
  'stamens',
  'saigō',
  'pelicans',
  'shelbourne',
  'fulmars',
  'cpu',
  'symbionts',
  'calculators',
  'placerville',
  'mashbir'])

In [150]:
imdb_not_wiki = imdb_words.difference(wiki_words)
len(imdb_not_wiki), [val for i, val in enumerate(itertools.islice(imdb_not_wiki, 20))]

(24264,
 ['rena',
  'rabal',
  'achmed',
  'unthinking',
  'grégoire',
  'puffing',
  'unwatched',
  'snotty',
  'film.it',
  'ghoulish',
  'senselessness',
  'rueful',
  'elysee',
  'campfest',
  'sacchi',
  'moya',
  'mohabbatein',
  'forklift',
  'boetticher',
  'paedophile'])

## 4.4 Out of vocab Initialization

คำว่า cpu อยู่ใน vocab ของ WikiText-103 แต่ไม่อยู่ใน IMDB

In [165]:
w = "cpu"
vocab.stoi[w], w in wiki_words, w in imdb_words

(0, True, False)

คำเหล่านี้ อยู่ใน vocab ของ IMDB แต่ไม่อยู่ใน WikiText-103 เราจะ Initialize Embedding ของคำเหล่านี้ แบบ Random

In [164]:
w = "senselessness"
vocab.stoi[w], w in wiki_words, w in imdb_words

(38905, False, True)

In [166]:
w = "unthinking"
vocab.stoi[w], w in wiki_words, w in imdb_words

(26254, False, True)

In [167]:
w = "forklift"
vocab.stoi[w], w in wiki_words, w in imdb_words

(43804, False, True)

Initialize ด้วยค่า Random เล็ก ๆ ใกล้เคียงกัน

In [168]:
np.allclose(enc.weight[vocab.stoi['senselessness'], :], 
            enc.weight[vocab.stoi['unthinking'], :])

True

แต่ถ้าเทียบกับ Embedding ที่เทรนเรียบร้อยแล้ว ก็จะต่างกันมาก

In [169]:
np.allclose(enc.weight[vocab.stoi['senselessness'], :], 
            enc.weight[vocab.stoi['capital'], :])

False

ดู Embedding

In [188]:
vocab.stoi['senselessness'], vocab.stoi['unthinking'], vocab.stoi['capital'], vocab.stoi['cpu']

(38905, 26254, 5727, 0)

In [189]:
len(enc.weight[vocab.stoi['senselessness'], :]), enc.weight[vocab.stoi['senselessness'], :][:50]

(400,
 tensor([ 0.0253,  0.0026,  0.0423, -0.0274, -0.0110, -0.0100, -0.0628,  0.1794,
         -0.1275, -0.2096, -0.0124, -0.0056, -0.0505, -0.0501, -0.0428,  0.1726,
         -0.0650, -0.0571, -0.1103,  0.0186,  0.0053, -0.0059,  0.0299,  0.0192,
         -0.0240,  0.1432, -0.1064, -0.0097,  0.0100, -0.0399,  0.0282,  0.0319,
          0.1297,  0.0687, -0.0047,  0.0168,  0.0201,  0.1346, -0.0336, -0.0449,
          0.0265, -0.0756, -0.0225,  0.0196,  0.0221,  0.0214,  0.0077, -0.1311,
         -0.1357, -0.0128], device='cuda:0', grad_fn=<SliceBackward>))

In [180]:
len(enc.weight[vocab.stoi['unthinking'], :]), enc.weight[vocab.stoi['unthinking'], :][:50]

tensor([ 0.0253,  0.0026,  0.0423, -0.0274, -0.0110, -0.0100, -0.0628,  0.1794,
        -0.1275, -0.2096, -0.0124, -0.0056, -0.0505, -0.0501, -0.0428,  0.1726,
        -0.0650, -0.0571, -0.1103,  0.0186,  0.0053, -0.0059,  0.0299,  0.0192,
        -0.0240,  0.1432, -0.1064, -0.0097,  0.0100, -0.0399,  0.0282,  0.0319,
         0.1297,  0.0687, -0.0047,  0.0168,  0.0201,  0.1346, -0.0336, -0.0449,
         0.0265, -0.0756, -0.0225,  0.0196,  0.0221,  0.0214,  0.0077, -0.1311,
        -0.1357, -0.0128], device='cuda:0', grad_fn=<SliceBackward>)

In [190]:
len(enc.weight[vocab.stoi['capital'], :]), enc.weight[vocab.stoi['capital'], :][:50]

(400,
 tensor([-0.2566,  0.1062,  0.3081,  0.2742,  0.1667, -0.1626, -0.5122, -0.1261,
          0.4548, -0.4177, -0.1565,  0.0752,  0.2253,  0.2031, -0.3125,  0.0310,
          0.5112, -0.3691, -0.2947, -0.0168, -0.1147,  0.3074, -0.1654, -0.2202,
          0.0538, -0.4041,  0.1666, -0.0109, -0.2803,  0.3044, -0.2279,  0.1786,
         -0.2502, -0.2334,  0.4438,  0.2404,  0.1746,  0.1509,  0.0351,  0.2844,
         -0.0244, -0.2031,  0.1228, -0.2357, -0.2065,  0.5811,  0.1338,  0.4087,
         -0.5034, -0.0150], device='cuda:0', grad_fn=<SliceBackward>))

# สรุป

* เราได้เรียนรู้ว่า 

# Credit

* https://www.youtube.com/watch?v=dt7sArnLo1g&list=PLtmWHNX-gukKocXQOkQjuVxglSDYWsSh9&index=6&t=0s
* https://www.bualabs.com/archives/2693/data-block-api-data-pipeline-machine-learning-supervised-learning-preprocessing-ep-5/
* https://www.bualabs.com/archives/926/sentiment-analysis-imdb-movie-review-ulmfit-sentiment-analysis-ep-1/
* https://www.bualabs.com/archives/3000/sentiment-classification-imdb-movie-reviews-with-naive-bayes-logistic-regression-nlp-ep-5/
* https://docs.fast.ai/data_block.html
* https://docs.fast.ai/text.data.html
* https://forums.fast.ai/t/language-model-zoo-gorilla/14623
* https://www.imdb.com/interfaces/
* https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/
* https://arxiv.org/abs/1801.06146
* 